In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import numpy as np
import pandas as pd
#import plotly.graph_objects as go
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
from tqdm import tqdm_notebook as tqdm
import pickle
from pathlib import Path
from nuscenes import NuScenes
from scipy.spatial.transform import Rotation as R 
from math import cos, sin, pi
from lyft_dataset_sdk.lyftdataset import *
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud, Box, Quaternion
from lyft_dataset_sdk.utils.geometry_utils import view_points, transform_matrix
from lyft_dataset_sdk.eval.detection.mAP_evaluation import Box3D, recall_precision

In [9]:
phase = 'train'
data = 'v1.0-trainval' if phase=='train' else 'v1.0-test'
path = '/media/jionie/my_disk/Kaggle/Lyft/input/3d-object-detection-for-autonomous-vehicles/train_root/'
nusc = NuScenes(dataroot=path, version=data, verbose=0) 
lyft=nusc

### prepare ground truth json files

Ground truth json files for train/val set are used for evaluation using  lyft's official mAP metric

In [5]:
# info_path = '../../../data/lyft/train/infos_val.pkl'
#info_path = '../../../data/lyft/train/infos_train.pkl'
# info_path = '../../../data/nuscenes/v1.0-trainval/infos_train.pkl'
info_path = '/media/jionie/my_disk/Kaggle/Lyft/input/3d-object-detection-for-autonomous-vehicles/train_root/infos_val.pkl'
with open(info_path, 'rb') as f:
    info = pickle.load(f)['infos']

In [6]:
# only for nuscenes
NameMapping = {
    'movable_object.barrier': 'barrier',
    'vehicle.bicycle': 'bicycle',
    'vehicle.bus.bendy': 'bus',
    'vehicle.bus.rigid': 'bus',
    'vehicle.car': 'car',
    'vehicle.construction': 'construction_vehicle',
    'vehicle.motorcycle': 'motorcycle',
    'human.pedestrian.adult': 'pedestrian',
    'human.pedestrian.child': 'pedestrian',
    'human.pedestrian.construction_worker': 'pedestrian',
    'human.pedestrian.police_officer': 'pedestrian',
    'movable_object.trafficcone': 'traffic_cone',
    'vehicle.trailer': 'trailer',
    'vehicle.truck': 'truck'
}


In [19]:
gt_data = [] 
for inf in tqdm(info):
    sample_token = inf['token']
    sample = lyft.get('sample', sample_token)
    for ann_token in sample['anns']:
        ann_record = lyft.get('sample_annotation', ann_token)
        try:
            data = {
                'sample_token': sample_token,
                'translation': ann_record['translation'],
                'size': ann_record['size'],
                'rotation': ann_record['rotation'],
                'name': ann_record['category_name']
#                 'name': NameMapping[ann_record['category_name']]
            }
            gt_data.append(data)
        except Exception as e:
            print(e) # for nuscenes, not using some categories

100%|██████████| 4158/4158 [00:00<00:00, 17227.02it/s]


In [21]:
len(gt_data), gt_data[0]

(113896,
 {'sample_token': 'f7fca7320eaf90ffc6d20bf5477a18ca61c042979eaaf0cda36603e1a969dad9',
  'translation': [1002.3932119976462, 2517.525612249236, -21.410812547321655],
  'size': [1.999, 4.817, 1.636],
  'rotation': [-0.9905516179554115, 0, 0, 0.13714041040450659],
  'name': 'car'})

In [22]:
# gt_data_path = '../../../data/lyft/train/gt_data_val.json'
# gt_data_path = '../../../data/lyft/train/gt_data_train.json'
# gt_data_path = '../../../data/nuscenes/v1.0-trainval/gt_data_train.json'
gt_data_path = '/media/jionie/my_disk/Kaggle/Lyft/input/3d-object-detection-for-autonomous-vehicles/train_root/gt_data_val.json'
with open(gt_data_path, 'w') as f:
    json.dump(gt_data, f, indent=2)